# ONE PAIR

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
cv2.__version__


In [ ]:
min_disparity=0
num_disp=160
sad_window_size=3
uniqueness=10
speckle_window_size=15
speckle_range=32
p1=216
p2=864
max_disparity=1
full_dp=False

In [ ]:
block_matcher = cv2.StereoSGBM(minDisparity=min_disparity,
                        numDisparities=num_disp,
                        SADWindowSize=sad_window_size,
                        uniquenessRatio=uniqueness,
                        speckleWindowSize=speckle_window_size,
                        speckleRange=speckle_range,
                        disp12MaxDiff=max_disparity,
                        P1=p1,
                        P2=p2,
                        fullDP=full_dp)

In [ ]:
left = cv2.imread('/root/data/blender_test/Image0028_L.png')[230:325, 370:650]
right = cv2.imread('/root/data/blender_test/Image0028_R.png')[230:325, 370:650]

In [ ]:
plt.imshow(left)
plt.show()

In [ ]:
plt.imshow(right)
plt.show()

In [ ]:
disp = block_matcher.compute(left, right).astype(np.float32) / 16.0

In [ ]:
plt.imshow(disp)
plt.show()

# TUNING

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
left = cv2.imread('/root/data/blender_test/Image0028_L.png')[230:325, 370:650]
right = cv2.imread('/root/data/blender_test/Image0028_R.png')[230:325, 370:650]
# left = cv2.resize(left, (800, 600))
# right = cv2.resize(right, (800, 600))

In [ ]:
params = dict()

for _ in range(50):
    
    mind = 0
    params['min_disparity']=mind
    params['num_disp']=np.random.choice(range(16, 320, 16))
    params['sad_window_size']=np.random.choice([3 ,5, 7, 9,11, 15, 17])
    params['uniqueness']=np.random.randint(5, 15)
    params['speckle_window_size']=np.random.randint(0, 50)
    params['speckle_range']=np.random.randint(0, 10)
    params['p1']=216
    params['p2']=864
    params['max_disparity']=np.random.randint(mind+1, mind+32)
    params['full_dp']=False
    print(params)
    block_matcher = cv2.StereoSGBM(minDisparity=params['min_disparity'],
                            numDisparities=params['num_disp'],
                            SADWindowSize=params['sad_window_size'],
                            uniquenessRatio=params['uniqueness'],
                            speckleWindowSize=params['speckle_window_size'],
                            speckleRange=params['speckle_range'],
                            disp12MaxDiff=params['max_disparity'],
                            P1=params['p1'],
                            P2=params['p2'],
                            fullDP=params['full_dp'])
    
    
    disp = block_matcher.compute(left, right).astype(np.float32) / 16.0
    f, ax = plt.subplots(1, 4, figsize=(20, 10))
    ax[0].imshow(left)
    ax[1].imshow(right)
    ax[2].imshow(disp)
    ax[3].imshow(left)
    ax[3].imshow(disp, alpha=0.2)
    plt.show()
    
    
    print('#'*100)

# good params

In [ ]:
params = {'p2': 864, 'p1': 216, 'speckle_window_size': 32, 'speckle_range': 6, 'min_disparity': 0, 'max_disparity': 20, 'uniqueness': 13, 'num_disp': 32, 'full_dp': False, 'sad_window_size': 7}
# params = {'p2': 864, 'p1': 216, 'speckle_window_size': 15, 'speckle_range': 7, 'min_disparity': 0, 'max_disparity': 14, 'uniqueness': 12, 'num_disp': 128, 'full_dp': False, 'sad_window_size': 7}
# params= {'p2': 864, 'p1': 216, 'speckle_window_size': 30, 'speckle_range': 3, 'min_disparity': 0, 'max_disparity': 25, 'uniqueness': 9, 'num_disp': 16, 'full_dp': False, 'sad_window_size': 7}

In [ ]:
left = cv2.imread('/root/data/blender_test/Image0028_L.png')[230:325, 370:650]
right = cv2.imread('/root/data/blender_test/Image0028_R.png')[230:325, 370:650]

In [ ]:
block_matcher = cv2.StereoSGBM(minDisparity=params['min_disparity'],
                        numDisparities=params['num_disp'],
                        SADWindowSize=params['sad_window_size'],
                        uniquenessRatio=params['uniqueness'],
                        speckleWindowSize=params['speckle_window_size'],
                        speckleRange=params['speckle_range'],
                        disp12MaxDiff=params['max_disparity'],
                        P1=params['p1'],
                        P2=params['p2'],
                        fullDP=params['full_dp'])

In [ ]:
disp = block_matcher.compute(left, right).astype(np.float32) / 16.0
f, ax = plt.subplots(1, 4, figsize=(20, 10))
ax[0].imshow(left)
ax[1].imshow(right)
ax[2].imshow(disp)
ax[3].imshow(left)
ax[3].imshow(disp, alpha=0.2)

In [ ]:
plt.imshow(disp)
plt.colorbar()
plt.show()

# overlay mask

In [ ]:
from utils import crop_and_mask
from PIL import Image

In [ ]:
mask = crop_and_mask(Image.open('/root/data/blender_test/Image0028_L.png').convert('RGB'))

In [ ]:
small_mask = mask[230:325, 370:650]

In [ ]:
plt.imshow(disp*small_mask)
plt.colorbar()
plt.show()

In [ ]:
disparities = disp*small_mask

In [ ]:
disparities.max()

In [ ]:
# meters
focal_length = 10.0*1e-3
baseline = 65.0*1e-3
image_sensor_width = 32.0*1e-3
image_sensor_height = 18.0*1e-3
pixel_size = image_sensor_width / 1024
print(pixel_size)

In [ ]:
focal_length_pixel = focal_length / pixel_size
depth = focal_length_pixel*baseline / disparities

In [ ]:
focal_length_pixel*baseline / 12

In [ ]:
depth[depth == np.inf] = 0
depth[depth == -np.inf] = 0

In [ ]:
plt.imshow(depth)
plt.colorbar()
plt.clim([0, 10])
plt.show()

In [ ]:
td = (np.load('/root/data/blender_test/true_depth.npy')*mask)[230:325, 370:650]
# td[td>14]=0
plt.imshow(td)
plt.colorbar()
plt.clim([1, 10])
plt.show()

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(20, 10))
im = ax[0].imshow(depth)
im2 = ax[1].imshow(td)
ax[0].set_title('prediction')
im.set_clim(1, 10)
im2.set_clim(1, 10)
ax[1].set_title('ground truth')

cbar_ax = f.add_axes([0.95, 0.30, 0.05, 0.5])
f.colorbar(im, cax=cbar_ax)
plt.show()

In [ ]:
total_err = []
xs, ys = np.nonzero(depth)
for (x,y) in zip(xs, ys):
    truth = td[x,y]
    pred = depth[x,y]
    err = np.abs(truth-pred) / truth
    total_err.append(err)

In [ ]:
np.mean(total_err)